<a href="https://colab.research.google.com/github/PrincetonUniversity/python_machine_learning/blob/main/notebook1_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

rng = np.random.RandomState(1)
qualitative_colors = ['#1b9e77','#d95f02','#7570b3','#e7298a']

# *Regression*

## Load Diabetes Data Set

In [ ]:
from sklearn.datasets import load_diabetes

In [ ]:
# Load the diabetes dataset
diabetes_bunch = load_diabetes()
diabetes_X = diabetes_bunch.data
diabetes_y = diabetes_bunch.target

# Use only one feature
diabetes_X = diabetes_X[:, np.newaxis, 2]

# Split the data into training/testing sets
diabetes_X_train = diabetes_X[:-20]
diabetes_X_test = diabetes_X[-20:]

# Split the targets into training/testing sets
diabetes_y_train = diabetes_y[:-20]
diabetes_y_test = diabetes_y[-20:]

In [ ]:
print(diabetes_bunch.DESCR)

In [ ]:
fig, axs = plt.subplots(figsize=(4.,4.), nrows=1, ncols=1, facecolor='white', dpi=200)
axs.scatter(diabetes_X_train, diabetes_y_train, color=qualitative_colors[0], s=10)
axs.set_xlabel('BMI (scaled)')
axs.set_ylabel('quantitative measure of diabetes progression')

## Linear Regression



In [ ]:
from sklearn.linear_model import LinearRegression

### Create linear regression object

In [ ]:
regr = LinearRegression()

### Train the model using the training set



In [ ]:
regr.fit(diabetes_X_train, diabetes_y_train)

### Output the best fit values

In [ ]:
print("Intercept: \n", regr.intercept_)
print("Coefficients: \n", regr.coef_)

In [ ]:
fig, axs = plt.subplots(figsize=(4.,4.), nrows=1, ncols=1, facecolor='white', dpi=200)
axs.scatter(diabetes_X_train, diabetes_y_train, s=1, color=qualitative_colors[0])
axs.plot(diabetes_X_train, regr.predict(diabetes_X_train), color=qualitative_colors[1], linewidth=2)
axs.scatter(diabetes_X_train[:4], diabetes_y_train[:4], color=qualitative_colors[0])
axs.vlines(diabetes_X_train[:4], regr.intercept_ + regr.coef_[0]*diabetes_X_train[:4], diabetes_y_train[:4], lw=2)
axs.set_xlabel('BMI (scaled)')
axs.set_ylabel('quantitative measure of diabetes progression')

### Make predictions using the test set

In [ ]:
diabetes_y_pred_linear = regr.predict(diabetes_X_test)

In [ ]:
fig, axs = plt.subplots(figsize=(4.,4.), nrows=2, ncols=1, facecolor='white', dpi=200, sharex=True)
axs[0].scatter(diabetes_X_train, diabetes_y_train, s=1, color=qualitative_colors[0])
axs[0].plot(diabetes_X_test, diabetes_y_pred_linear, color=qualitative_colors[1], linewidth=2)
axs[0].scatter(diabetes_X_test, diabetes_y_test, color=qualitative_colors[2], s=8)
axs[1].hlines(0, -0.1, 0.15, color=qualitative_colors[1], linewidth=2)
axs[1].scatter(diabetes_X_test, diabetes_y_test-diabetes_y_pred_linear, color=qualitative_colors[2], s=10)
axs[1].set_xlabel('BMI (scaled)')
axs[0].set_ylabel('diabetes progression')
axs[1].set_ylabel('truth - model')

## Exercise

Try applying the linear regression model to data on laptop size versus price:

In [ ]:
import pandas as pd
df = pd.read_csv("https://tigress-web.princeton.edu/~jdh4/laptop.csv")
plt.scatter(df["Inches"], df["Price (Euro)"])
plt.xlabel("Size in Inches")
plt.ylabel("Price (Euro)")

In [ ]:
laptop_X = df["Inches"].values.reshape(-1, 1)
laptop_y = df["Price (Euro)"].values

In [ ]:
# begin exercse here (start by creating laptop_X_train, laptop_X_test, laptop_y_train, laptop_y_test)

## Create Periodic Data Set (Optional)

In [ ]:
X = np.linspace(start=0, stop=2.*np.pi, num=1000).reshape(-1, 1)
y_base = np.squeeze(np.sin(X))

noise_std = 0.5
y = y_base + rng.normal(loc=0.0, scale=noise_std, size=y_base.shape)

training_indices = rng.choice(np.arange(y.size), size=10, replace=False)
X_train, y_train = X[training_indices], y[training_indices]

In [ ]:
fig, axs = plt.subplots(figsize=(4.,4.), nrows=1, ncols=1, facecolor='white', dpi=200, sharex=True)
axs.plot(X, y_base, color="gray", label = r"$y = \sin(x)$")
axs.errorbar(X_train, y_train, noise_std, linestyle="None", color=qualitative_colors[0], marker=".", markersize=10)
axs.legend()
axs.set_ylabel('y')
axs.set_xlabel('x')

## Gaussian Process Regression

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor

### Create kernel object

In [ ]:
from sklearn.gaussian_process.kernels import RBF
kernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-4, 1e2))

### Create Gaussian Process object

In [ ]:
gaussian_process = GaussianProcessRegressor(kernel=kernel, alpha=noise_std**2, n_restarts_optimizer=9)

### Train the model using the training set

In [ ]:
gaussian_process.fit(X_train, y_train)

### Make predictions

In [ ]:
mean_prediction, std_prediction = gaussian_process.predict(X, return_std=True)

In [ ]:
fig, axs = plt.subplots(figsize=(4.,4.), nrows=2, ncols=1, facecolor='white', dpi=200, sharex=True)
axs[0].plot(X, y_base, color="gray")
axs[0].errorbar(X_train, y_train, noise_std, linestyle="None", color=qualitative_colors[0], marker=".", markersize=10)
axs[0].plot(X, mean_prediction, ls='--')
axs[0].fill_between(X.ravel(), mean_prediction - 1.96 * std_prediction, mean_prediction + 1.96 * std_prediction, alpha=0.25)
axs[0].set_ylabel('y')
axs[1].set_xlabel('x')
axs[1].plot(X, np.zeros(X.shape), color="gray")
axs[1].plot(X, y_base-mean_prediction, ls='--')
axs[1].fill_between(X.ravel(), (y_base-mean_prediction) - 1.96 * std_prediction, (y_base-mean_prediction) + 1.96 * std_prediction, alpha=0.25)
axs[1].set_ylabel('truth - model')